In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import pickle

# Step 1: Đọc triplet và tạo ánh xạ
df_triplet = pd.read_csv("f_coco_triplets.csv")
entities = pd.concat([df_triplet["subject"], df_triplet["object"]]).unique()
entity2id = {entity: idx for idx, entity in enumerate(entities)}
relation2id = {rel: idx for idx, rel in enumerate(df_triplet["predicate"].unique())}

# Lưu ánh xạ
with open("entity2id.pkl", "wb") as f:
    pickle.dump(entity2id, f)
with open("relation2id.pkl", "wb") as f:
    pickle.dump(relation2id, f)

# Step 2: Gán ID cho triplets
df_triplet["head_id"] = df_triplet["subject"].map(entity2id)
df_triplet["tail_id"] = df_triplet["object"].map(entity2id)
df_triplet["rel_id"] = df_triplet["predicate"].map(relation2id)
triplets = df_triplet[["head_id", "rel_id", "tail_id"]].values

# Step 3: Định nghĩa mô hình TransH
class TransH(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim=100, margin=1.0):
        super().__init__()
        self.margin = margin
        self.ent_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.rel_embeddings = nn.Embedding(num_relations, embedding_dim)
        self.norm_embeddings = nn.Embedding(num_relations, embedding_dim)
        nn.init.xavier_uniform_(self.ent_embeddings.weight)
        nn.init.xavier_uniform_(self.rel_embeddings.weight)
        nn.init.xavier_uniform_(self.norm_embeddings.weight)

    def _project(self, e, norm):
        norm = F.normalize(norm, p=2, dim=-1)
        return e - torch.sum(e * norm, dim=-1, keepdim=True) * norm

    def forward(self, pos_triples, neg_triples):
        ph = self.ent_embeddings(pos_triples[:, 0])
        pr = self.rel_embeddings(pos_triples[:, 1])
        pt = self.ent_embeddings(pos_triples[:, 2])
        pn = self.norm_embeddings(pos_triples[:, 1])

        nh = self.ent_embeddings(neg_triples[:, 0])
        nr = self.rel_embeddings(neg_triples[:, 1])
        nt = self.ent_embeddings(neg_triples[:, 2])
        nn_ = self.norm_embeddings(neg_triples[:, 1])

        ph_proj = self._project(ph, pn)
        pt_proj = self._project(pt, pn)
        nh_proj = self._project(nh, nn_)
        nt_proj = self._project(nt, nn_)

        pos_score = torch.norm(ph_proj + pr - pt_proj, p=2, dim=1)
        neg_score = torch.norm(nh_proj + nr - nt_proj, p=2, dim=1)

        return pos_score, neg_score

# Step 4: Huấn luyện mô hình
def corrupt_batch(batch, num_entities):
    corrupted = batch.copy()
    for i in range(len(batch)):
        if np.random.rand() < 0.5:
            corrupted[i][0] = np.random.randint(0, num_entities)
        else:
            corrupted[i][2] = np.random.randint(0, num_entities)
    return corrupted

def train_model(model, train_data, num_entities, batch_size=512, lr=0.001, epochs=300):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MarginRankingLoss(margin=model.margin)
    model.train()

    for epoch in range(epochs):
        np.random.shuffle(train_data)
        total_loss = 0

        for i in range(0, len(train_data), batch_size):
            pos_batch = train_data[i:i+batch_size]
            neg_batch = corrupt_batch(pos_batch.copy(), num_entities)

            pos_batch = torch.tensor(pos_batch, dtype=torch.long)
            neg_batch = torch.tensor(neg_batch, dtype=torch.long)
            y = torch.ones(pos_batch.size(0))

            pos_score, neg_score = model(pos_batch, neg_batch)
            loss = criterion(pos_score, neg_score, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

    torch.save(model.state_dict(), "transh_model.pth")
    return model

model = TransH(num_entities=len(entity2id), num_relations=len(relation2id))
train_model(model, triplets, num_entities=len(entity2id))

# Step 5: Lưu embedding
entity_emb = model.ent_embeddings.weight.detach().cpu().numpy()
relation_emb = model.rel_embeddings.weight.detach().cpu().numpy()
np.save("entity_embeddings.npy", entity_emb)
np.save("relation_embeddings.npy", relation_emb)

Epoch 1: Loss = 6.9233
Epoch 2: Loss = 6.7290
Epoch 3: Loss = 6.5307
Epoch 4: Loss = 6.3209
Epoch 5: Loss = 6.1294
Epoch 6: Loss = 5.9450
Epoch 7: Loss = 5.7262
Epoch 8: Loss = 5.4979
Epoch 9: Loss = 5.2914
Epoch 10: Loss = 5.0357
Epoch 11: Loss = 4.8294
Epoch 12: Loss = 4.6041
Epoch 13: Loss = 4.3904
Epoch 14: Loss = 4.1842
Epoch 15: Loss = 4.0101
Epoch 16: Loss = 3.8447
Epoch 17: Loss = 3.7278
Epoch 18: Loss = 3.5192
Epoch 19: Loss = 3.4311
Epoch 20: Loss = 3.3440
Epoch 21: Loss = 3.1814
Epoch 22: Loss = 3.0008
Epoch 23: Loss = 2.9855
Epoch 24: Loss = 2.9706
Epoch 25: Loss = 2.8416
Epoch 26: Loss = 2.7769
Epoch 27: Loss = 2.7230
Epoch 28: Loss = 2.6188
Epoch 29: Loss = 2.5830
Epoch 30: Loss = 2.5822
Epoch 31: Loss = 2.5454
Epoch 32: Loss = 2.4507
Epoch 33: Loss = 2.4049
Epoch 34: Loss = 2.3026
Epoch 35: Loss = 2.3853
Epoch 36: Loss = 2.3329
Epoch 37: Loss = 2.2391
Epoch 38: Loss = 2.2497
Epoch 39: Loss = 2.2175
Epoch 40: Loss = 2.1963
Epoch 41: Loss = 2.2250
Epoch 42: Loss = 2.1372
E

In [2]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
from tqdm import tqdm

# Load lại ánh xạ
with open("entity2id.pkl", "rb") as f:
    entity2id = pickle.load(f)
with open("relation2id.pkl", "rb") as f:
    relation2id = pickle.load(f)

# Load embeddings và ảnh
entity_emb = np.load("entity_embeddings.npy")
relation_emb = np.load("relation_embeddings.npy")
image_emb = np.load("image_features.npy")
image_ids = np.load("image_ids.npy")

# Load triplet và ánh xạ ảnh → entity
df = pd.read_csv("f_coco_triplets.csv")
img_entity_map = defaultdict(set)
for _, row in df.iterrows():
    img_id = str(row["image_id"])
    if row["subject"] in entity2id:
        img_entity_map[img_id].add(entity2id[row["subject"]])
    if row["object"] in entity2id:
        img_entity_map[img_id].add(entity2id[row["object"]])

# Đánh giá từng triplet
def evaluate_single_triplet(h, r, t, top_k=5):
    pred_vec = entity_emb[h] + relation_emb[r]
    scores = []
    for idx, img_id in enumerate(image_ids):
        entities_in_img = img_entity_map.get(str(img_id), set())
        if not entities_in_img:
            continue
        dists = [np.linalg.norm(pred_vec - entity_emb[eid]) for eid in entities_in_img]
        scores.append((img_id, min(dists)))
    top_k_imgs = sorted(scores, key=lambda x: x[1])[:top_k]
    return set(str(img[0]) for img in top_k_imgs)

# Tổng hợp GT để đánh giá
gt_dict = defaultdict(set)
for _, row in df.iterrows():
    key = (row["subject"], row["predicate"], row["object"])
    gt_dict[key].add(str(row["image_id"]))

# Đánh giá toàn bộ test set
total_p = total_r = total_f1 = 0
count = 0
for key in tqdm(gt_dict):
    s, p, o = key
    if s not in entity2id or o not in entity2id or p not in relation2id:
        continue
    h, r, t = entity2id[s], relation2id[p], entity2id[o]
    pred = evaluate_single_triplet(h, r, t, top_k=5)
    true = gt_dict[key]
    if not true:
        continue
    tp = len(pred & true)
    fp = len(pred - true)
    fn = len(true - pred)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    total_p += precision
    total_r += recall
    total_f1 += f1
    count += 1

print(f"\n Precision@5: {total_p/count:.4f}")
print(f" Recall@5:    {total_r/count:.4f}")
print(f" F1-score@5:  {total_f1/count:.4f}")

100%|███████████████████████████████████████████████████████████████████████████| 2613/2613 [00:00<00:00, 48009.34it/s]


 Precision@5: 0.0000
 Recall@5:    0.0000
 F1-score@5:  0.0000
